In [2]:
import sqlite3
from bs4 import BeautifulSoup
import requests, csv

In [3]:
#make list of Gatogeries

url = "https://books.toscrape.com/"
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
gategories = soup.find('div', attrs={'class','side_categories'}).find('ul').find('ul').find_all('a')

#insert Gatogeries to database

conection = sqlite3.connect('Books.db')
for gategory in gategories:
    Data = [gategory.get_text().strip()]
    sql = f'INSERT INTO Gategories (GategoryName) VALUES (?)'
    cursor = conection.execute(sql,Data)
    conection.commit()
conection.close()

In [4]:
rates = {'One': 1, 'Two': 2, 'Three': 3, 'Four': 4, 'Five': 5}
pages = ['index', 'page-2', 'page-3', 'page-4', 'page-5', 'page-6', 'page-7', 'page-8']

In [5]:
# insert Books to database

for gategory in gategories:

    # number of books in the page

    url_before = f"http://books.toscrape.com/{gategory.attrs['href']}"
    response = requests.get(url_before)
    soup = BeautifulSoup(response.text, 'html.parser')
    num = (int(soup.find_all('strong')[1].get_text()) // 20) + 1
    GategoryId = gategories.index(gategory) + 1

    # loop for each page 

    for i in range(num):
        url_after= url_before[:-10]
        url = url_after + pages[i] + '.html'
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        all_articles = soup.find_all('article', attrs={'class': 'product_pod'})

        for article in all_articles:
            title = article.find('h3').find('a').get('title')
            price = float(article.find('p', attrs={'class': 'price_color'}).get_text().split('£')[1])
            rating = rates[article.find('p').get('class')[1]]
            conection = sqlite3.connect("Books.db") 
            sql = f'INSERT INTO Books (Title, Price, Rate,GategoryId) VALUES (?, ?, ?, ?)'
            Data = [title, price,rating, GategoryId]
            cursor = conection.execute(sql,Data)
            conection.commit()
        conection.close()         
        